## Data visualization (and statistics)

- create mock data for the experiments 
- make visualizations
    - bar graph
(- statistical test )

In [ ]:
# imports

# data manipulation and processing
import pandas as pd
import numpy as np

# visualizations
from bokeh.models import FactorRange, Legend
from bokeh.palettes import Colorblind, ColumnDataSource
from bokeh.plotting import figure, show, output_notebook

output_notebook()


### create mock data fingerprint experiment

In [ ]:
rows = ['1', '2', '3', '4', '5', '6']
columns = ['HS1-C1', 'HS1-C2', 'HS1-C3', 'HS1-1', 'HS1-2', 'HS1-3', 'HS2-C1', 'HS2-C2', 'HS2-C3', 'HS2-1', 'HS2-2', 'HS2-3']

df = pd.DataFrame(np.random.randint(0, 150, size=(6, 12)), columns=columns, index=rows)

In [ ]:
df

In [ ]:
# get the mean for each triplicate
df['HS1_S'] = df[df.columns[df.columns.str.contains('HS1-\d+')]].mean(axis=1)
df['HS1_C'] = df[df.columns[df.columns.str.contains('HS1-C')]].mean(axis=1)
df['HS2_S'] = df[df.columns[df.columns.str.contains('HS2-\d+')]].mean(axis=1)
df['HS2_C'] = df[df.columns[df.columns.str.contains('HS2-C')]].mean(axis=1) 

df_res = df.iloc[:, -4:].T
df_res['count'] = df_res[df_res.columns].mean(axis=1)

df_res

In [ ]:
df_res.reset_index(inplace=True)
df_res[['HS', 'sample_type']] = df_res['index'].str.split('_', n=1, expand=True)
df_res.drop(['index'], axis=1, inplace=True)

## visualization

In [ ]:
df_res['sample_type'] = df_res['sample_type'].map({'S': 'treated', 'C': 'control'})
df_res['x'] = df_res[['HS', 'sample_type']].apply(lambda x: (x[0],str(x[1])), axis=1)
df_res

In [ ]:
# nested bar graph: https://stackoverflow.com/questions/67901133/create-nested-bar-graph-in-bokeh-from-a-dataframe
# legend append: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

p = figure(
    x_range=FactorRange(*list(df_res["x"].unique())),
    width=500
)

factors = df_res['sample_type'].unique()

# Manually specify colors for each factor
colors = Colorblind[3][:len(factors)]

legend_items = []
for i, factor in enumerate(factors):
    source = df_res[df_res['sample_type'] == factor]
    vbar = p.vbar(x='x', top='count', width=0.9, source=source,
                  color=colors[i])
    legend_items.append((factor, [vbar]))

p.y_range.start = 0
p.y_range.end = df_res['count'].max() * 1.2
p.x_range.range_padding = 0.25

p.title = "Number of colonies per handsanitizer"
p.title.text_font_size = '15px'
p.yaxis.axis_label = "Number of colonies"
p.xaxis.axis_label = "Hand Sanitizers"
p.xgrid.grid_line_color = None

# Create a legend
legend = Legend(items=legend_items, location="top_center")
legend.label_text_font_size = "12px"
legend.spacing = 5
legend.click_policy = "hide"  

p.add_layout(legend, 'below')

# Show the plot
show(p)
